<h1 style="color:#65AE11;">使用多个 GPU 执行复制和计算重叠</h1>

在本节中，你将学习通过必要的索引策略在多个 GPU 上实现复制和计算的重叠。

<h2 style="color:#65AE11;">目标</h2>

完成本节内容的学习后，您将能理解：

* 流与每个 GPU 设备是如何关联的
* 如何为多个 GPU 创建非默认流
* 如何在多个 GPU 上执行复制和计算重叠

<h2 style="color:#65AE11;">流和多 GPU</h2>

每个 GPU 都有各自的默认流。我们也可以为当前处于活动状态的 GPU 设备创建、使用和销毁非默认流。切记不要在未与当前处于活动状态的 GPU 建立关联的流中启动核函数。

<h2 style="color:#65AE11;">为多个 GPU 创建多个流</h2>

在多个 GPU 上使用多个非默认流时，我们可以将这些流存储在二维数组中，使得数组中的每一行皆包含单个 GPU 的所有的流：

```c
cudaStream_t streams[num_gpus][num_streams]; // 2D array containing number of streams for each GPU.

// For each available GPU...
for (uint64_t gpu = 0; gpu < num_gpus; gpu++) {
    // ...set as active device...
    cudaSetDevice(gpu);
    for (uint64_t stream = 0; stream < num_streams; stream++)
        // ...create and store its number of streams.
        cudaStreamCreate(&streams[gpu][stream]);
}
```

<h2 style="color:#65AE11;">多个 GPU 上多流的数据块大小</h2>

当在多个 GPU 上使用多个非默认流时，全局数据索引尤为棘手。为此，我们可以为单个流和整个 GPU 分别定义数据块大小：

```c
// Each stream needs num_entries/num_gpus/num_streams data. We use round up division for
// reasons previously discussed.
const uint64_t stream_chunk_size = sdiv(sdiv(num_entries, num_gpus), num_streams);

// It will be helpful to also to have handy the chunk size for an entire GPU.
const uint64_t gpu_chunk_size = stream_chunk_size*num_streams;
```

<h2 style="color:#65AE11;">为多个 GPU 的多个流分配显存</h2>

GPU 显存并未分配给各个流，所以此处的分配操作看起来与之前的多 GPU 任务相似，我们只需注意数据块的大小是分配给整个 GPU 的而非其中一个流的即可：

```c
// For each GPU...
for (uint64_t gpu = 0; gpu < num_gpus; gpu++) {

    // ...set device as active...
    cudaSetDevice(gpu);

    // ...use a GPU chunk's worth of data to calculate indices and width...
    const uint64_t lower = gpu_chunk_size*gpu;
    const uint64_t upper = min(lower+gpu_chunk_size, num_entries);
    const uint64_t width = upper-lower;

    // ...allocate data.
    cudaMalloc(&data_gpu[gpu], sizeof(uint64_t)*width);
}
```

<h2 style="color:#65AE11;">为多个 GPU 的多个流执行复制和计算重叠</h2>

我们将在每个GPU上的多个非默认流中执行数据复制与计算的重叠。此技术与仅使用单个 GPU 的情况非常相似，所以我们只需在遍历每个 GPU 时应用单GPU的实现方法，并额外注意数据索引即可。请慢慢地体会这一部分。

```c
// For each GPU...
for (uint64_t gpu = 0; gpu < num_gpus; gpu++) {
    // ...set device as active.
    cudaSetDevice(gpu);
    // For each stream (on each GPU)...
    for (uint64_t stream = 0; stream < num_streams; stream++) {

        // Calculate index offset for this stream's chunk of data within the GPU's chunk of data...
        const uint64_t stream_offset = stream_chunk_size*stream;
        
        // ...get the lower index within all data, and width of this stream's data chunk...
        const uint64_t lower = gpu_chunk_size*gpu+stream_offset;
        const uint64_t upper = min(lower+stream_chunk_size, num_entries);
        const uint64_t width = upper-lower;

        // ...perform async HtoD memory copy...
        cudaMemcpyAsync(data_gpu[gpu]+stream_offset, // This stream's data within this GPU's data.
                        data_cpu+lower,              // This stream's data within all CPU data.
                        sizeof(uint64_t)*width,      // This stream's chunk size worth of data.
                        cudaMemcpyHostToDevice,
                        streams[gpu][stream]);       // Using this stream for this GPU.

        kernel<<<grid, block, 0, streams[gpu][stream]>>>    // Using this stream for this GPU.
            (data_gpu[gpu]+stream_offset,                   // This stream's data within this GPU's data.
             width);                                        // This stream's chunk size worth of data.

        cudaMemcpyAsync(data_cpu+lower,              // This stream's data within all CPU data.
                        data_gpu[gpu]+stream_offset, // This stream's data within this GPU's data.
                        sizeof(uint64_t)*width,
                        cudaMemcpyDeviceToHost,
                        streams[gpu][stream]);       // Using this stream for this GPU.
    }
}
```

<h2 style="color:#65AE11;">知识检查</h2>

请回答下列问题，确认您已了解本节的主要目标。您可以通过单击问题下方的`...`单元来显示每个问题的答案。

---

**以下哪项表述是正确的？请选择所有适用项。**

1. 默认流专用于索引为 `0` 的 GPU 设备
2. 每个 GPU 都有各自的默认流
3. 我们可以为当前处于活动状态的 GPU 创建非默认流
4. 我们可使用单个非默认流在多个 GPU 上执行计算

**答案：2、3**

---

**在与当前未处于活动状态的 GPU 相关联的流中启动核函数后，我们即可在当前未处于活动状态的 GPU 上启动核函数。**

1. 正确
2. 错误

**答案：2**

如果在未与当前处于活动状态的 GPU 设备建立关联的流中发布核函数，此核函数将无法启动。

---

<h2 style="color:#65AE11;">下一步</h2>

你现在已经掌握在多个 GPU 上执行复制和计算重叠所需的全部技术。下一节中，你还将再次重构密码应用，在多个 GPU 上开展计算时执行复制和计算重叠，从而实现大幅加速。

请继续学习下一节：[*多个 GPU 与复制和计算重叠*](../08_Exercise_MGPU_Streams/Exercise_MGPU_Streams.ipynb)。